<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module4/M4_1/1_Integer_Project_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming: Modeling Logic Constraints (Project Selection)
**OPIM 5641: Business Decision Modeling - University of Connecticut**

Please see Powell Chapter 11 (Integer Optimization) for more details and examples.

------------------------------------------------------------------------


# Introduction
A binary variable, which takes on the values zero or one, can be used to represent a 'go/no go' decision. We can think in terms of discrete projects, where the decision to accept the project is represented by the value '1' and the decision to reject the project is represented by the value '0'. Let's examine problems that involve this **'binary choice'**.

In [ ]:
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

# Example: Project Selection

A company has been allocated \$160 million  for projects this year. There are five projects that can be selected, each with an initial cost to complete the project and a value (NPV) that comes from its implementation. The projects cover a wide range of activites:
* P1: Implement a new IT system
* P2: License a new technology from another firm
* P3: Build a state-of-the-art recycling facility
* P4: Install an automated machining center in production
* P5: Move the receiving department to new facilties on site.

There is just one project of each type. Each project has an estimated NPV and each requires a capital expenditure, which must come out of the budget for capital projects. The following table summarizes the possibilities, with all ﬁgures in millions of dollars:

Project | P1 | P2 | P3 | P4 | P5 |
--- | --- | --- | --- | --- | --- |
NPV | 10 | 17 | 16 | 8 | 14 |
Expenditure| 48 | 96 | 80 | 32 | 64

You have to select from the 5 possible projects to work on. Your objective is to maximize NPV (value for each project given below) and to completely exhaust your budget for projects. Challenge accepted!

# First Attempt: allocation model with one constraint.
We can formulate this problem as an allocation model with one constraint. To construct an algebraic model, we let

<center>
$y_j = 1$ if project $j$ is accepted; and $0$ otherwise.
</center>

The decision variables represent binary choice. When $y_j = 1$, project $j$ is selected to be part of the set of projects that are undertaken. When $yj = 0$, project $j$ is rejected. Only these two binary values have meaning as decisions.

**Objective function:**

<center>
$\max(z) = 10y_1 + 17y_2 + 16y_3 + 8y_4 + 14y_5$
</center>

... subject to...

**Constraints:** 

<center>
$48y_1 + 96y_2 + 80y_3 + 32y_4 + 64y_5 \leq 160$
</center>

Let's code this thing up in Pyomo!


In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables
model.y1 = Var(domain=NonNegativeIntegers) # non-negative integers!
model.y2 = Var(domain=NonNegativeIntegers) # 
model.y3 = Var(domain=NonNegativeIntegers) # 
model.y4 = Var(domain=NonNegativeIntegers) # 
model.y5 = Var(domain=NonNegativeIntegers) # 

# 3) objective function: maximize value!
model.OBJ = Objective(expr = 10*model.y1 + 17*model.y2 + 16*model.y3 + 8*model.y4 + 14*model.y5,
                      sense = maximize)

# 4) constraints
model.Constraint1 = Constraint(expr = 48*model.y1 + 96*model.y2 + 80*model.y3 + 32*model.y4 + 64*model.y5 <= 160)

# # these are unncessary if you force a binary variable type!
# # you would only need these if you used 'NonNegativeIntegers' (you could, but more work/typing/errors...)
# model.ConstraintP1 = Constraint(expr = model.y1 <= 1)
# model.ConstraintP2 = Constraint(expr = model.y2 <= 1)
# model.ConstraintP3 = Constraint(expr = model.y3 <= 1)
# model.ConstraintP4 = Constraint(expr = model.y4 <= 1)
# model.ConstraintP5 = Constraint(expr = model.y5 <= 1)

# check it out, then run it!
model.pprint()

5 Var Declarations
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    y5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 10*y1 +

Run it!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 40.0
  Upper bound: 40.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 5
  Number of binary variables: 0
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      

Evaluate the output.

In [ ]:
# show the results
print("Objective = ", model.OBJ())
print("Project P1 = ", model.y1())
print("Project P2 = ", model.y2())
print("Project P3 = ", model.y3())
print("Project P4 = ", model.y4())
print("Project P5 = ", model.y5())

Objective =  40.0
Project P1 =  0.0
Project P2 =  0.0
Project P3 =  0.0
Project P4 =  5.0
Project P5 =  0.0


In [ ]:
# check out the constraint
print('Budget Constraint =', model.Constraint1()) # we spent the budget 32*5 = 160

Budget Constraint = 160.0


In [ ]:
32*5

160

**OH NO!** We've created a \$40M in value, but we've selected this project P4 five times, which is a very silly thing to do. Each of these projects can only happen once (re-written here for clarity). So, you shouldn't use `NonNegativeIntegers` - you need to use `Binary`.

* P1: Implement a new IT system
* P2: License a new technology from another firm
* P3: Build a state-of-the-art recycling facility
* P4: Install an automated machining center in production
* P5: Move the receiving department to new facilties on site.

Let's try this again but now add the constraint that you can only select each project once.

# Second Attempt: each project can only be selected once.
Ensure that each project is only selected once. Rather than writing a constraint that our variables $y_1$ through $y_5$ be less than or equal to 1, let's force them to take on `Binary` values. This implicitly handles this constraint.



In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables
model.y1 = Var(domain=Binary) # takes on values of 0/1
model.y2 = Var(domain=Binary) 
model.y3 = Var(domain=Binary) 
model.y4 = Var(domain=Binary) 
model.y5 = Var(domain=Binary) 

# 3) objective function
model.OBJ = Objective(expr = 10*model.y1 + 17*model.y2 + 16*model.y3 + 8*model.y4 + 14*model.y5,
                      sense = maximize)

# 4) constraints
model.Constraint1 = Constraint(expr = 48*model.y1 + 96*model.y2 + 80*model.y3 + 32*model.y4 + 64*model.y5 <= 160)

# # these are unncessary if you force a binary variable type!
# # you would only need these if you used 'NonNegativeIntegers' (you could, but more work/typing/errors...)
# model.ConstraintP1 = Constraint(expr = model.y1 <= 1)
# model.ConstraintP2 = Constraint(expr = model.y2 <= 1)
# model.ConstraintP3 = Constraint(expr = model.y3 <= 1)
# model.ConstraintP4 = Constraint(expr = model.y4 <= 1)
# model.ConstraintP5 = Constraint(expr = model.y5 <= 1)

# check it out, then run it!
model.pprint()

5 Var Declarations
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 10*y1 + 17*y2 + 16*y3 + 8*y4 + 14*y5

1 Constraint Declarations
    Cons

Solve it!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 34.0
  Upper bound: 34.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 5
  Number of binary variables: 5
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
    

Review it!

In [ ]:
# show the results
print("Objective = ", model.OBJ())
print("Project P1 = ", model.y1())
print("Project P2 = ", model.y2())
print("Project P3 = ", model.y3())
print("Project P4 = ", model.y4())
print("Project P5 = ", model.y5())

Objective =  34.0
Project P1 =  1.0
Project P2 =  0.0
Project P3 =  1.0
Project P4 =  1.0
Project P5 =  0.0


In [ ]:
# check out the constraint
print('Budget Constraint =', model.Constraint1()) # we spent the budget 48 + 80 + 32

Budget Constraint = 160.0


In [ ]:
48 + 80 + 32

160

**Much better!** We spent our budget, but of course, by introducing more constraints, we do not improve the optimal solution. But now at least we have a feasible solution where we create a value of \$34 million.

A brief note: our presentation follows slightly differently from the book - if you were to use `NonNegativeReals` instead of `Binary` you would have ended up with an objective of \$35.2 million, but you would have had fractional projects (which can't happen!) Go ahead and run this and you'll see why it's a problem! How can you do 0.2 of a project?! The model will always run - it's up to you to ensure that you have given the appropriate boundaries/parameters!

It is important that you use `Binary` variables when it calls for it - we will keep the naming convention of using `x` for continuous (`NonNegativeReals`) variables and `y` for binary (`Binary`) variables.

# Binary Variables and Logical Relationships
Projects can be related in any number of ways. We cover ﬁve types of relationships here: 
* At least m projects must be selected. 
* At most n projects must be selected. 
* Exactly k projects must be selected. 
* Some projects are mutually exclusive. 
  * If P1 is selected, then you cannot select P2.
* Some projects have contingency relationships.
  * If P1 is selected, then you must select P2.


## At least m projects must be selected
Suppose now that projects P2 and P5 are international projects, while the others are domestic. Suppose also that the committee wishes to select **at least one** of its projects from the international arena. We can then add a covering constraint to the base case:

<center>
$y_2 + y_5 \geq 1$
</center>

Because the variables are all binary, this constraint ensures that the combination

<center>
$y_2 = y_5 = 0$ `they both can't be 0 at the same time!`
</center>

is not allowed. Therefore, P2, or P5, or both, will be selected, thus satisfying the requirement of one international selection.

This can just be added as a constraint in our model...

`model.Constraint_Intl = Constraint(expr = model.y2 + model.y5 >= 1)`

If we run this example, we will find that NPV drops to $\text{\$32M}$, we only spend $\text{\$144M}$ of the budget, and we select projects $P1$, $P4$ and $P5$. Go ahead and try this out!

Other notes:
* This is known as a 'covering constraint' because you must meet the constraint (think of the nutrition example where we called that a 'covering' problem, meeting our minimum standards).
* We have found that *we can use our binary decision variables to help describe policy actions*. Let's see another example below.


## At most n projects must be selected
Here's a different policy action. Imagine the company can only provide oversight support on two or less projects. Then we could write:

<center>
$y_1 + y_2 + y_3 + y_4 + y_5 \leq 2$
</center>

This ensures two or less projects might be chosen.

## Exactly k projects must be selected
Similary, perhaps the board of trustees at the company wants to have exactly two big projects to hang their hats on.

<center>
$y_1 + y_2 + y_3 + y_4 + y_5 = 2$
</center>

A simple tweak of the symbol yields a different policy action.

## Some projects are mutually exclusive
Other relationships that we normally think of as “logical” relationships can also be expressed with binary variables. Suppose that projects $P4$ and $P5$ are **mutually exclusive** (for example, they could require some of the same staff resources). 

Then we could write:
<center>
$y_4 + y_5 \leq 1$
</center>

This constraint prohibits the combination:
<center>
$y_4 = y_5 = 1$ `they both can't be 1 at the same time!`
</center>

Another example might be cloud software migration - we want to do an overhaul of our IT system and switch to Amazon ($P4$) or Microsoft ($P5$). You would do either one of these (or none of these!) but not both!


## Some projects have contingency relationships
On the flip side, imagine that you are an e-commerce company. Project $P3$ might be build a new warehouse for Amazon goods, and Project $P5$ might be overhaul the payment/web interface for Amazon. You can imagine that you wouldn't do one of these without the other!

**Contingency relationships** also invoke another term: **consistency**. 

Suppose that project $P5$ requires that $P3$ be selected. In other words, $P5$ is contingent on $P3$.

To analyze logical requirements of this sort, consider all of the selection combinations for the binary variables. The following table shows that three of the four combinations are **consistent** (valid) with the contingency condition:

y3|y5|Consistent?
---|---|---
0|0|Yes
1|0|Yes
0|1|No
1|1|Yes

Three of the options are consistent/valid with the contingency constraint, and one is not. We can address the inconsistent constraint as follows:

<center>
$y_3 - y_5 \geq 0$
</center>

Note how the order matters here! We said that $P5$ can only happen if $P3$ occurs, but not vice-versa (we didn't say that we couldn't just do $P3$ - in fact, this is consistent!)

# Third Attempt: Putting it all together
Let's try incorporating one of each of these constraints.

In [ ]:
# 1) concrete model
model = ConcreteModel()

# 2) declare decision variables
model.y1 = Var(domain=Binary)
model.y2 = Var(domain=Binary) 
model.y3 = Var(domain=Binary) 
model.y4 = Var(domain=Binary) 
model.y5 = Var(domain=Binary) 

# 3) objective function
model.OBJ = Objective(expr = 10*model.y1 + 17*model.y2 + 16*model.y3 + 8*model.y4 + 14*model.y5,
                      sense = maximize)

# 4) constraints
model.Constraint1 = Constraint(expr = 48*model.y1 + 96*model.y2 + 80*model.y3 + 32*model.y4 + 64*model.y5 <= 160)

# at least one of the 2 projects must be selected 
# imagine y2 and y5 are international projects that need one to be selected
model.Constraint2 = Constraint(expr = model.y2 + model.y5 >=1)

# at most 4 projects must be selected
model.Constraint3 = Constraint(expr = model.y1 + model.y2 + model.y3 + model.y4 + model.y5 <= 4)

# mutually exclusive projects
# imagine you can only pick P4 or P5 (or neither of them), but not both of them
model.Constraint4 = Constraint(expr = model.y4 + model.y5 <= 1)

# contingency: write a constraint for the inconsistent constraint.
# if you pick P5, you MUST do P3. 
# you can pick P3 and not do P5.
model.Constraint5 = Constraint(expr = model.y3 - model.y5 >= 0)

# check it out, then run it!
model.pprint()

5 Var Declarations
    y1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y4 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    y5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 10*y1 + 17*y2 + 16*y3 + 8*y4 + 14*y5

5 Constraint Declarations
    Cons

Solve it!

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 30.0
  Upper bound: 30.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 5
  Number of binary variables: 5
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      

Review it!

In [ ]:
# show the results
print("Objective = ", model.OBJ())
print("Project P1 = ", model.y1())
print("Project P2 = ", model.y2())
print("Project P3 = ", model.y3())
print("Project P4 = ", model.y4())
print("Project P5 = ", model.y5())

Objective =  30.0
Project P1 =  0.0
Project P2 =  0.0
Project P3 =  1.0
Project P4 =  0.0
Project P5 =  1.0


In [ ]:
# check out the constraint
print('Budget Constraint =', model.Constraint1()) # we only spent 144M

Budget Constraint = 144.0


There you have it! **By adding more and more constraints, the value of your objective function DID NOT IMPROVE!!!**

# [Optional] Appendix
If you want to raise your math and Python skills, check out this implementation.

**Define the Objective Function**

$\max(z) = \sum\limits_{i = 1}^{5}p_iy_i$

**Write the Constraints**

subject to:

* $0 \leq y_i  \leq 1, i \in [5]$ 

In [ ]:
# declare the model
model = ConcreteModel()

# Data - you can't index on the 0,
# so you need to provide a 0 (seems silly, but needed)
projects = np.arange(6) # for our 5 projects (does not include 6)
NPV = [0, 10, 17, 16, 8, 14]
Cost = [0, 48, 96, 80, 32, 64]

# declare decision variables
model.y = Var(projects, domain=Binary)

# objective function
obj_expr = 0
for project in projects:
  obj_expr += NPV[project]*model.y[project]

# declare objective
model.profit = Objective(
                      expr = obj_expr, # values come from the table
                      sense = maximize)

In [ ]:
# declare constraints -> we will populate the list of constraints step by step
model.constraints = ConstraintList()

# Constraint 1: Expenditures
model.constraints.add(Cost[1]*model.y[1] + 
                      Cost[2]*model.y[2] + 
                      Cost[3]*model.y[3] + 
                      Cost[4]*model.y[4] + 
                      Cost[5]*model.y[5] <= 160) 

# Constraint 2: at least 2 projects must be selected 
# imagine y2 and y5 are international projects that need one to be selected
model.constraints.add(model.y[2] + model.y[5] >= 1)

# Constraint 3: at most 4 projects must be selected
model.constraints.add(model.y[1] + 
                      model.y[2] + 
                      model.y[3] + 
                      model.y[4] + 
                      model.y[5] <= 4) 

# Constraint 4: mutually exclusive projects
# imagine you can only pick P4 or P5 (or neither of them), but not both of them
model.constraints.add(model.y[4] + model.y[5] <= 1)

# Constraint 5:
# contingency: write a constraint for the inconsistent constraint.
# if you pick P5, you MUST do P3. 
# you can pick P3 and not do P5.
model.constraints.add(model.y[3] - model.y[5] >= 0)

# show the model you've created
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    y : Size=6, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 10*y[1] + 17*y[2] + 16*y[3] + 8*y[4

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("Total profit = ", model.profit())
for project in projects:
  print("Project",project,":",model.y[project]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 30.0
  Upper bound: 30.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 5
  Number of binary variables: 5
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      

Same answer as before! Both are good approaches.